In [109]:
# This script shows you how to make a submission using a few
# useful Python libraries.
# It gets a public leaderboard score of 0.76077.
# Maybe you can tweak it and do better...?

import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import LabelEncoder

In [110]:
#get the training dataset
raw_data = pd.read_csv('train.csv')

In [111]:
#drop few columns
data = raw_data.drop(['Name','Ticket','Cabin','PassengerId'],axis=1)

In [95]:
#investigating data

data = data.dropna()


In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [113]:
#set missing age

#data["Age"].fillna(data.groupby("Pclass")["Age"].transform("mean"), inplace=True)

In [114]:
def setIsChild(age):
    if age < 10:
        return 1
    else:
        return 0
#data.head()

In [115]:
IsChild = data.Age.apply(setIsChild)
data['IsChild'] = IsChild
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
IsChild     891 non-null int64
dtypes: float64(2), int64(5), object(2)
memory usage: 62.7+ KB


In [116]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 9 columns):
Survived    889 non-null int64
Pclass      889 non-null int64
Sex         889 non-null object
Age         889 non-null float64
SibSp       889 non-null int64
Parch       889 non-null int64
Fare        889 non-null float64
Embarked    889 non-null object
IsChild     889 non-null int64
dtypes: float64(2), int64(5), object(2)
memory usage: 69.5+ KB


In [117]:

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data.Sex = le.fit_transform(data.Sex)
data.Embarked = le.fit_transform(data.Embarked)

In [118]:
X=data.drop('Survived',axis=1)
y = data['Survived']

In [119]:
#train all the models
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()

In [120]:
#ensemble with voting = hard
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X, y)


In [121]:
# formatting test data
temp = pd.read_csv('test.csv')
temp = temp.PassengerId



In [122]:
X_test = pd.read_csv('test.csv')
X_test = X_test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
X_test["Age"].fillna(X_test.groupby("Pclass")["Age"].transform("mean"), inplace=True)

X_test["Fare"].fillna(X_test.groupby("Pclass")["Fare"].transform("mean"), inplace=True)

age = X_test.Age.apply(setIsChild)
X_test['IsChild'] = age
X_test.Sex = le.fit_transform(X_test.Sex)
X_test.Embarked = le.fit_transform(X_test.Embarked)

In [123]:
predictions = eclf1.predict(X_test)

In [124]:
output = pd.DataFrame({'PassengerId':temp,'Survived':predictions})
#ensemble voting
output.to_csv('titanic_output_EnsembleVoting.csv',index=False)